In [22]:
# https://huggingface.co/turhancan97?sort_models=modified#models
# https://huggingface.co/models?other=yolov5&sort=trending
import yolov5

import smtplib as smt
from password import password
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os 
from jinja2 import Environment, FileSystemLoader
from datetime import datetime



class GarbichCunter():
    """ 
    Class for counting garbage in a beach video feed and sending alerts.
    
    Attributes:
        link (str): URL of the video feed.
        results, predictions, boxes, scores, categories, count: Variables to store detection results.
        city_name, beach_name, video_url: Variables for location details and video URL.
    """
    
    def __init__(self, link, conf=0.7, iou=0.5, agnostic=False, multi_label=False, max_det = 100):
        """ 
        Initializes the GarbichCunter with video link and model parameters.
        
        Args:
            link (str): URL of the video feed.
            conf (float): Confidence threshold for the model.
            iou (float): Intersection over Union threshold for the model.
            agnostic (bool): Class-agnostic setting for the model.
            multi_label (bool): Multi-label setting for the model.
            max_det (int): Maximum number of detections per image.
        """
        
        self.link = link
        # Load the garbage detection model
        model = yolov5.load('keremberke/yolov5s-garbage')
        
        # Set model parameters
        model.conf = conf  # Set NMS confidence threshold
        model.iou = iou  # Set NMS IoU threshold
        model.agnostic = agnostic  # Set NMS class-agnostic
        model.multi_label = multi_label  # Set NMS multiple labels per box
        model.max_det = max_det  # Set maximum number of detections per image

        # Perform inference with test time augmentation
        self.results = model(link, size=1240, augment=True)

        # Parse results from the model
        self.predictions = self.results.pred[0]
        self.boxes = self.predictions[:, :4] # Coordinates of bounding boxes
        self.scores = self.predictions[:, 4] # Confidence scores
        self.categories = self.predictions[:, 5] # Categories of detected objects
        self.count = len(self.scores) # Count of detected objects
    
    def send_message(self, email_getter, threshold, video_url, city_name=' ', beach_name=' '):
        """
        Sends an email alert if the count of detected garbage exceeds the threshold.
        
        Args:
            email_getter (str): Recipient's email address.
            threshold (int): Threshold count to trigger alert.
            video_url (str): URL of the video for reference.
            city_name (str): Name of the city where the beach is located.
            beach_name (str): Name of the beach.
        """
        self.city_name = city_name
        self.beach_name = beach_name
        self.video_url = video_url
        
        # Check if garbage count exceeds the threshold
        if self.count >= threshold:
            
            email_sender = 'clean_beach_alert@mail.ru'
            host_sender = email_sender.split('@')[1]

            # Connect to SMTP server
            smtp_server = smtplib.SMTP(f'smtp.{host_sender}', 587)
            # Start encryption
            smtp_server.starttls()
            
            msg = MIMEMultipart()
            # Set subject of the email
            msg['Subject'] = 'Important notification from CBA on Beach Condition'
            
            # Load the current directory as template directory
            file_loader = FileSystemLoader(os.getcwd())
            # Create Environment object for template handling
            env = Environment(loader=file_loader)  
            
            # Load HTML email template
            template = env.get_template('template.html')

            # Data to replace in the template
            data = {
                "city_name": self.city_name,
                "date": datetime.now().strftime("%d %B %Y"),
                "beach_name": self.beach_name,
                "video_url": self.video_url
            }

            # Render HTML message with data
            html_mess = template.render(data)

            msg.attach(MIMEText(html_mess, 'html'))


            smtp_server.login(email_sender, pasword)
            smtp_server.sendmail(email_sender, email_getter, msg.as_string())

            smtp_server.quit()     

In [23]:
gar = GarbichCunter('/Users/zond/Desktop/python/vs_code/ipnd_les/pec/beach_test_3.png', conf=0.20)

gar.send_message('pwp50336@zbock.com', 3, 'https://10minutemail.net/', 'MSK', 'prod')

4